This notebook encodes a video with each frame contains blocks of black and white that represent binary data.

In [1]:
in_path = 'input'  # path to an arbitrary file you want to encode.
out_path = 'output'  # path to the encoded .mp4 video file.

W, H = 1920, 1080  # width and height of each frame.
S = 4  # squared size of each block.

assert W % S == 0 and H % S == 0, 'width and height need to be divisible by squared size'
assert W % 4 == 0, 'better to have 4-byte alignment to avoid generating padding'

BPF = W*H//(S*S*8) # number of bytes per frame

In [3]:
from PIL import Image
import numpy as np

import subprocess
import shlex
import itertools

def to_binary(b):
    a = np.frombuffer(b, 'uint8')
    return np.vstack([a >> 7-i & 1 for i in range(8)]).T.reshape(-1)

In [6]:
args = shlex.split(f'ffmpeg -framerate 30 -f image2pipe -i - -c:v libx264 -r 30 -pix_fmt yuv420p "{out_path}"')
ffmpeg = subprocess.Popen(args, stdin=subprocess.PIPE, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL).stdin

try:
    with open(in_path, 'rb') as f:
        for count in itertools.count(1):
            data = f.read(BPF)
            if not data:
                break

            if len(data) != BPF:
                data += bytes(BPF - len(data))

            x = (to_binary(data) * 255).reshape(H // S, W // S)
            im = Image.fromarray(x).resize((W, H), resample=0)
            im.save(ffmpeg, 'bmp')
            
            print(f'Frames Encoded: {count}', end='\r')
finally:
    ffmpeg.close()

print('\nDONE!')

Frames Encoded: 615
DONE!
